In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'

import os
import sys
import time
from multiprocessing import Process, Pool
import warnings
warnings.simplefilter('ignore')

# Add module path to system path
module_paths = ['..', 
                '../..',
                '../../extern/PsrPopPy',
                # '/Users/vohl/opt/miniconda3/lib/python3.9/site-packages'
                # '/Users/vohl/miniconda3/lib/python3.10/site-packages'
               ]
for module_path in module_paths:
    if os.path.abspath(os.path.join(module_path)) not in sys.path:
        sys.path.insert(0, module_path)
    
    
# For convenience
import numpy as np
import pickle

import pandas as pd

from epn_mining.main import load_states

from epn_mining.utils.io import load, save

from epn_mining.preparation import epn
from epn_mining.preparation.pulsar import Population, Observation, Pulsar, Model, Component
from epn_mining.topology import topology
from epn_mining.analysis.stats import (
    centroid as compute_centroid,
    profile_as_distribution,
    evaluate_DPGMM,
    convert_x_to_phase
)
from epn_mining.analysis import stats
from epn_mining.analysis.distance import (check_bound, check_min_max, check_neg)

from epn_mining.preparation.signal import (
    shift_max_to_center, 
    shift_centroid_to_center,
    rotate,
    best_alignment
)

from epn_mining.analysis import plotting

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits import axes_grid1
from matplotlib import rc
from matplotlib import style, collections as mc, colors, cm

style.use('default')

style.use('default')

import json
from sklearn import mixture
from scipy.stats import norm
from joblib import parallel_backend
from tqdm import tqdm

# from dtaidistance import dtw

from dtw import dtw

import astropy.coordinates as coord
import astropy.units as u
from astropy.coordinates import SkyCoord

import copy

# Set session variables
verbose = True
state_store=True

pink = (230/255, 29/255, 95/255, 1)
blue = (47/255, 161/255, 214/255, 0.2)
blue_full = (47/255, 161/255, 214/255, 1)

cmap = cm.get_cmap('cubehelix').reversed()

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
# state_prefix='paper_stokes_I'
# state_prefix = 'paper_not_normalized_stokes_I_only'
state_prefix = 'paper'

epn_metadata = load('epn_metadata', state_prefix=state_prefix)
population = load('population', state_prefix=state_prefix)

In [3]:
metric = 'DTW'
stokes_to_include = [ 'model' ]
weights = {k:w for k,w in zip(stokes_to_include, [1, 0.5, 0.5])}
freq_ids_to_include = [2,3,4,5]

_4freqs_ = True if len(freq_ids_to_include) > 1 else False

# freq_ids_to_include = [2,4]
min_snr = 20

from dtw import rabinerJuangStepPattern

penalty = None
step_pattern = 'asymmetric' # rabinerJuangStepPattern(6, "d") # 'symmetric2'#'symmetricP05' #['', 'symmetric2']
window_type  = 'sakoechiba' #['sakoechiba', None]
window_args  = {'window_size': 256} #{'window_size': 512} #[{'window_size':204}, {}]
open_begin   = False #[False, True, True]
open_end     = False #[False, True, True]

# state_name = None
state_name = '%sbin%s_graph_%s%s_%s' % (len(freq_ids_to_include) if _4freqs_ else '',
                                             '' if _4freqs_ else 's' + str(freq_ids_to_include[0]),
                                             step_pattern, 
                                             '_%s' % window_type if window_type is not None else '',                 
                                             stokes_to_include[0]
                                            )

cropped = False
verbose = True

print ('state_name', state_name)

state_name 4bin_graph_asymmetric_sakoechiba_model


In [15]:
distances, \
population_graph_indices, \
graph_population_indices = topology.complete_undirected_weighted_graph(population,
                                                                       metric=metric,
                                                                       stokes_to_include=stokes_to_include,
                                                                       weights=weights,
                                                                       freq_ids_to_include=freq_ids_to_include,
                                                                       min_snr=min_snr,
                                                                       penalty=penalty,
                                                                       step_pattern=step_pattern,
                                                                       window_type=window_type,
                                                                       window_args=window_args,
                                                                       open_begin=open_begin,
                                                                       open_end=open_end,
                                                                       cropped=cropped,
                                                                       verbose=verbose)

Make weighted graph


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 95/95 [04:21<00:00,  2.75s/it]

complete graph: |e| = 4560


In [17]:
save(state_name, #'4bins_graph', 
     [distances, population_graph_indices, graph_population_indices], 
     protocol=pickle.DEFAULT_PROTOCOL,
     state_prefix=state_prefix)

In [16]:
for v, n in zip(
    [distances, population_graph_indices, graph_population_indices],
    ['distances', 'population_graph_indices', 'graph_population_indices'],
):
    save(n, 
         v, 
         protocol=pickle.DEFAULT_PROTOCOL,
         state_prefix=state_prefix)

In [6]:
mst = topology.spanning_tree(distances, type='minimum', verbose=verbose)
sequence_population, sequence_indices, mst = topology.longest_path(mst,
                                                                   population.as_array(),
                                                                   graph_population_indices,
                                                                   verbose=verbose)
elongation, normalized_elongation  =  mst.elongation()

elongation_dict = {}
elongation_dict[state_prefix] = {'elongation' : elongation,
                                 'normalized_elongation' : normalized_elongation,
                                 'length': mst.length(),
                                 'half_width': mst.half_width(),
                                 'longest_path' : len(sequence_population),
                                 'N' : mst.V}

Compute minimum spanning tree
Compute longest path in minimum spanning tree


In [ ]:
save(state_name, 
     [distances, population_graph_indices, graph_population_indices], 
     protocol=pickle.DEFAULT_PROTOCOL,
     state_prefix=state_prefix)